## Imports

In [4]:
import pandas as pd
import numpy as np
import json
import requests
from datetime import datetime

Mock Question 1: Transaction Risk Scoring
Prompt:
You’re given a list of transactions (like what a fraud model might score). Each transaction has:

amount

country

risk_score (0–100)

Write a function flagged_transactions(transactions) that returns a list of transaction IDs that should be flagged if:

1) risk_score > 80

2) OR amount > 5000 and country != "US"

In [18]:
transactions = [
    {"id": "tx1", "amount": 1000, "country": "US", "risk_score": 50},
    {"id": "tx2", "amount": 6000, "country": "UK", "risk_score": 40},
    {"id": "tx3", "amount": 3000, "country": "US", "risk_score": 90}
]

In [20]:
transactions

[{'id': 'tx1', 'amount': 1000, 'country': 'US', 'risk_score': 50},
 {'id': 'tx2', 'amount': 6000, 'country': 'UK', 'risk_score': 40},
 {'id': 'tx3', 'amount': 3000, 'country': 'US', 'risk_score': 90}]

In [22]:
#Manual Example First

In [24]:
transactions[0]

{'id': 'tx1', 'amount': 1000, 'country': 'US', 'risk_score': 50}

In [28]:
transactions[0]["risk_score"]

50

In [32]:
list_of_flagged_IDs = []

for obj in transactions:

    #Check if should flagged or not
    if (obj["risk_score"] > 80) or (obj["amount"] > 5000 and obj["country"] != "US"): 
        list_of_flagged_IDs.append(obj["id"])

list_of_flagged_IDs

['tx2', 'tx3']

In [36]:
#Input: JSON response transactions
#Output: List of flagged transactions IDs
def flagged_transactions(transactions):

    list_of_flagged_IDs = []

    for obj in transactions:

        #Check if should flagged or not
        if (obj["risk_score"] > 80) or (obj["amount"] > 5000 and obj["country"] != "US"): 
            list_of_flagged_IDs.append(obj["id"])
            
            
    return list_of_flagged_IDs    

In [38]:
flagged = flagged_transactions(transactions)
flagged

['tx2', 'tx3']

Mock Question 2: Aggregate Income by Employer
Prompt:
Given a list of income sources (like Plaid might return), compute the total monthly income per employer.

Write a function aggregate_income(sources) that returns a dictionary where:

keys = employer name

values = total monthly income for that employer

In [40]:
sources = [
    {"employer": "Google", "monthly_income": 4000},
    {"employer": "Uber", "monthly_income": 1200},
    {"employer": "Google", "monthly_income": 1000}
]

In [42]:
sources

[{'employer': 'Google', 'monthly_income': 4000},
 {'employer': 'Uber', 'monthly_income': 1200},
 {'employer': 'Google', 'monthly_income': 1000}]

In [48]:
df = pd.DataFrame(sources)
df

,employer,monthly_income
0,Google,4000
1,Uber,1200
2,Google,1000


In [52]:
out = df.groupby("employer")["monthly_income"].sum()
out.to_dict()

{'Google': 5000, 'Uber': 1200}

In [54]:
#Input: JSON response sources
#Output Dictionary {"employer_name":"total_mo_inc"}


def aggregate_income(sources):

    agg_inc_dict = {}
    
    df = pd.DataFrame(sources) #convery dict to DF
    out = df.groupby("employer")["monthly_income"].sum() #groupby employer and sum over monthly_income

    agg_inc_dict = out.to_dict()

    return agg_inc_dict
    

In [105]:
aggregate_income(sources)

{'Google': 5000, 'Uber': 1200}

In [119]:
test = group_incomes_by_employer(sources)

test

{'Google': [4000, 1000], 'Uber': [1200]}

In [117]:
###Another method

def group_incomes_by_employer(sources):
    groups = {}

    for obj in sources:
        employer = obj["employer"]
        groups.setdefault(employer, []).append(obj["monthly_income"])

    return groups

Mock Question 3: Suspicious Login Detection
Prompt:
You're given a list of login events. Each event contains:

user_id

timestamp (in "YYYY-MM-DD HH:MM" format)

ip_address

country

Write a function flag_suspicious_logins(events) that returns a list of user_ids who:

Logged in from 2 or more different countries within the same 24-hour period.



In [61]:
events = [
    {"user_id": "u1", "timestamp": "2025-07-28 08:00", "ip_address": "1.1.1.1", "country": "US"},
    {"user_id": "u2", "timestamp": "2025-07-28 09:00", "ip_address": "2.2.2.2", "country": "US"},
    {"user_id": "u1", "timestamp": "2025-07-28 15:00", "ip_address": "3.3.3.3", "country": "FR"},
    {"user_id": "u3", "timestamp": "2025-07-29 10:00", "ip_address": "4.4.4.4", "country": "US"}
]

In [65]:
df = pd.DataFrame(events) #convert dict to DF
df

,user_id,timestamp,ip_address,country
0,u1,2025-07-28 08:00,1.1.1.1,US
1,u2,2025-07-28 09:00,2.2.2.2,US
2,u1,2025-07-28 15:00,3.3.3.3,FR
3,u3,2025-07-29 10:00,4.4.4.4,US


In [99]:
counts = df.groupby("user_id")["country"].nunique()

counts_df = pd.DataFrame(counts)

df2 = counts_df[(counts_df["country"] >= 1)]
df2

list_from_df = df2.values.tolist()
list_from_df

[[2], [1], [1]]